In [ ]:
import pandas as pd

df = pd.read_csv('C:/Users/OWNER PC/OneDrive/Documents/ML Projects/ADNOC Dataset/ADNOC_Stock_historical_Data.csv')
df.head()

In [ ]:
print(df.info())
print(df.describe())

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

#sns.pairplot(df[['Open', 'High', 'Low', 'Close', 'Volume']])
df[['Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change', 'Percentage_Change', 'Average_Price', 'Range']].hist(bins=60, figsize=(12,8))

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 20))

columns = ['Open', 'High', 'Low', 'Close', 'Volume']

for i in range(5):
    for j in range(5):
        if i != j:
            axs[i, j].scatter(df[columns[i]], df[columns[j]])
            axs[i, j].set_xlabel(columns[i])
            axs[i, j].set_ylabel(columns[j])
        else:
            axs[i, j].hist(df[columns[i]], bins=50)
            axs[i, j].set_title(columns[i])

plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df.corr()
print(corr_matrix)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True, fmt='.2f')

import numpy as np

corr_no_diagonal = np.triu(corr_matrix, k=1)
max_corr_idx = np.unravel_index(np.argmax(np.abs(corr_matrix)), corr_no_diagonal.shape)
max_corr = corr_no_diagonal[max_corr_idx]

plt.scatter(max_corr_idx[1], max_corr_idx[0], marker='*', s=200, c='red')
plt.text(max_corr_idx[1], max_corr_idx[0], f'Corr: {max_corr:.2f}\nFeatures: {corr_matrix.columns[max_corr_idx[1]]} vs {corr_matrix.columns[max_corr_idx[0]]}', ha='center', va='center', fontsize=10)

plt.title('Correlation Matrix')
plt.show()

In [ ]:
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Open', 'Close', 'High', 'Low', 'Volume', 'Price_Change', 'Percentage_Change', 'Average_Price', 'Range']] = scaler.fit_transform(df[['Open', 'Close', 'High', 'Low', 'Volume', 'Price_Change', 'Percentage_Change', 'Average_Price', 'Range']])

df['Date'] = pd.to_datetime(df['Date'])

df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

df.drop('Date', axis=1, inplace=True)
from sklearn.model_selection import train_test_split

X = df.drop('Average_Price', axis=1)
y = df['Average_Price']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print('Preprocessing Complete')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import mean_squared_error

model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=[mean_squared_error], steps_per_execution=False)

history = model.fit(X_train, y_train, epochs=100, batch_size=42, validation_data=(X_test, y_test))

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

plt.plot(history.history['val_mean_squared_error'], label='Validation Mean Squared Error')
plt.legend()
plt.show()

mse = model.evaluate(X_test, y_test)
rmse = np.sqrt(mse)

print('RMSE: ', rmse[0])